In [1]:
from random import random, randint, choice
from copy import deepcopy
from math import log

In [20]:
# 一个封装类，对应于“函数型”节点上的函数
class fwrapper:
    # function 函数本身
    # childcount 参数个数
    # name 函数名称
    def __init__(self, function, childcount, name):
        self.function = function
        self.childcount = childcount
        self.name = name

# 对应于函数型节点（即带子节点的节点）
class node:
    def __init__(self, fw, children):
        self.function = fw.function
        self.name = fw.name
        self.children = children
    
    # 当 evaluate 被调用时，对各个子节点进行求值运算，再将函数本身应用于求得的结果
    def evaluate(self, inp):
        results = [n.evaluate(inp) for n in self.children]
        return self.function(results)
    
    def display(self, indent=0):
        print((' '*indent) + self.name)
        for c in self.children:
            c.display(indent+1)
    
# 参数节点
# 该类对应的节点只返回传递给程序的某个参数
class paramnode:
    def __init__(self, idx):
        self.idx = idx
    
    def evaluate(self, inp):
        return inp[self.idx]
    
    def display(self, indent=0):
        print('%sp%d' % (' '*indent, self.idx))

# 常量节点
# 返回常量值的节点
class constnode:
    def __init__(self, v):
        self.v = v
        
    def evaluate(self, inp):
        return self.v
    
    def display(self, indent=0):
        print('%s%d' % (' '*indent, self.v))

In [21]:
# 针对节点的操作函数
addw = fwrapper(lambda l:l[0]+l[1], 2, 'add')
subw = fwrapper(lambda l:l[0]-l[1], 2, 'subtract')
mulw = fwrapper(lambda l:l[0]*l[1], 2, 'multiply')

def iffunc(l):
    if l[0] > 0:
        return l[1]
    else:
        return l[2]
    
ifw = fwrapper(iffunc, 3, 'if')

def isgreater(l):
    if l[0] > l[1]:
        return 1
    else:
        return 0

gtw = fwrapper(isgreater, 2, 'isgreater')

flist = [addw, mulw, ifw, gtw, subw]

In [22]:
# def func(x,y):
#     if x > 3:
#         return y + 5
#     else:
#         return y - 2
def exampletree():
    return node(ifw, [
        node(gtw, [paramnode(0), constnode(3)]),
        node(addw, [paramnode(1), constnode(5)]),
        node(subw, [paramnode(1), constnode(2)])
    ])

# abs(x+y)
# def abs(x,y):
#     if x+y > 3:
#         return x + y
#     else:
#         return -(x+y)
def abstree():
    return node(ifw, [
        node(gtw, [
            node(addw, [
                paramnode(0), 
                paramnode(1)
            ]),
            constnode(0)
        ]),
        node(addw, [
            paramnode(0), 
            paramnode(1)
        ]),
        node(subw, [
            constnode(0),
            node(addw, [
                paramnode(0),
                paramnode(1)
            ])      
        ])
    ])

In [23]:
exampletree = exampletree()
print(exampletree.evaluate([2,3]))
print(exampletree.evaluate([5,3]))
exampletree.display()

1
8
if
 isgreater
  p0
  3
 add
  p1
  5
 subtract
  p1
  2


In [24]:
abstree = abstree()
print(abstree.evaluate([2,3]))
print(abstree.evaluate([2,-3]))
abstree.display()

5
1
if
 isgreater
  add
   p0
   p1
  0
 add
  p0
  p1
 subtract
  0
  add
   p0
   p1


In [27]:
# pc 参数数量
# maxdepth 最大深度
# fpr 产生的节点为函数节点的概率
# ppr 产生的节点为参数节点的概率
def makerandomtree(pc, maxdepth=4, fpr=0.5, ppr=0.6):
    if random() < fpr and maxdepth > 0:
        f = choice(flist)
        children = [makerandomtree(pc, maxdepth-1, fpr, ppr) for i in range(f.childcount)]
        return node(f, children)
    
    elif random() < ppr:
        return paramnode(randint(0, pc-1))
    else:
        return constnode(randint(0, 10))

In [29]:
random1 = makerandomtree(2)
print(random1.evaluate([7,1]))
print(random1.evaluate([2,4]))


7
2


In [30]:
random1.display()

subtract
 p0
 0


In [31]:
random2 = makerandomtree(2)
print(random2.evaluate([5,3]))
print(random2.evaluate([5,20]))
random2.display()

4
-13
add
 subtract
  6
  p1
 isgreater
  subtract
   p1
   subtract
    2
    p1
  2


In [32]:
def hiddenfunction(x,y):
    return x**2 + 2*y + 3*x + 5

def buildhiddenset():
    rows = []
    for i in range(200):
        x = randint(0, 40)
        y = randint(0, 40)
        rows.append([x,y,hiddenfunction(x,y)])
    return rows

hiddenset = buildhiddenset()

In [33]:
def scorefunction(tree, s):
    dif = 0
    for data in s:
        v = tree.evaluate([data[0], data[1]])
        dif += abs(v-data[2])
    return dif

In [34]:
scorefunction(random2, hiddenset)

134314

In [35]:
scorefunction(random1, hiddenset)

127623

In [36]:
# 以用一棵全新的树来替换子树的方式进行变异
def mutate(t, pc, probchange=0.1):
    if random() < probchange:
        return makerandomtree(pc)
    else:
        result = deepcopy(t)
        if isinstance(t, node):
            result.children = [mutate(c, pc, probchange) for c in t.children]
        return result

In [39]:
random2.display()
muttree = mutate(random2, 2)
print('')
muttree.display()

add
 subtract
  6
  p1
 isgreater
  subtract
   p1
   subtract
    2
    p1
  2

add
 subtract
  6
  p1
 isgreater
  subtract
   p1
   subtract
    p1
    p1
  2


In [40]:
print(scorefunction(random2, hiddenset))
print(scorefunction(muttree, hiddenset))

134314
134314


In [41]:
def crossover(t1, t2, probswap=0.7, top=1):
    if random() < probswap and not top:
        return deepcopy(t2)
    else:
        result = deepcopy(t1)
        if isinstance(t1, node) and isinstance(t2, node):
            result.children = [crossover(c, choice(t2.children), probswap, 0) for c in t1.children]
        
        return result

In [53]:
random1 = makerandomtree(2)
random1.display()
print('\n')
random2 = makerandomtree(2)
random2.display()
print('\n')
cross = crossover(random1, random2)
cross.display()

multiply
 isgreater
  add
   if
    p0
    3
    4
   isgreater
    5
    p1
  if
   p0
   add
    0
    3
   p1
 isgreater
  7
  p1


isgreater
 if
  multiply
   p0
   p0
  subtract
   if
    p1
    p0
    p0
   subtract
    p1
    p1
  10
 p0


multiply
 if
  multiply
   p0
   p0
  subtract
   if
    p1
    p0
    p0
   subtract
    p1
    p1
  10
 p0


In [59]:
# rankfunction 将一组程序从优到劣进行排序的函数
# mutationrate 变异概率
# breedingrate 重组概率
# popsize 初始种群的大小
# probexp 选择劣势程序概率的递减比率
# probnew 引入一个全新的随机程序的概率
def evolve(pc, popsize, rankfunction, maxgen=500, mutationrate=0.1, breedingrate=0.4, pexp=0.7, pnew=0.05):
    # 返回一个随机数，通常是较小的数
    # pexp 的取值越小，得到的随机数就越小
    def selectindex():
        return int(log(random()) / log(pexp))
    # 创建一个随机的初始种群
    population = [makerandomtree(pc) for i in range(popsize)]
    for i in range(maxgen):
        scores = rankfunction(population)
        print(scores[0][0])
        if scores[0][0] == 0:
            break
        
        # 总能得到两个最优的程序
        # 精英选拔法（保送）
        newpop = [scores[0][1], scores[1][1]]
        # 构造下一代
        while len(newpop) < popsize:
            if random() > pnew:
                newpop.append(mutate(
                    crossover(scores[selectindex()][1],
                             scores[selectindex()][1],
                             probswap=breedingrate),
                    pc, probchange=mutationrate
                ))
            else:
                # 加入一个随机节点，以增加种群的多样性
                newpop.append(makerandomtree(pc))
                
        population = newpop
    
    scores[0][1].display()
    return scores[0][1]

def getrankfunction(dataset):
    def rankfunction(population):
        # 获取列表的第一个元素
        def takeFirst(elem):
            return elem[0]
        
        scores = [(scorefunction(t, dataset), t) for t in population]
        scores.sort(key=takeFirst)
        return scores
    
    return rankfunction

In [60]:
rf = getrankfunction(buildhiddenset())
evolve(2, 500, rf, mutationrate=0.2, breedingrate=0.1, pexp=0.7, pnew=0.1)

24672
7547
5886
5750
5088
3070
2867
2101
1000
0
add
 add
  p1
  5
 subtract
  p1
  subtract
   p0
   multiply
    p0
    add
     p0
     4


In [61]:
def gridgame(p):
    # 游戏区域的大小
    Max = (3,3)
    # 记住每位玩家的上一步
    lastmove = [-1,-1]
    # 记住玩家的位置
    location = [[randint(0, Max[0]), randint(0, Max[1])]]
    # 将第二位玩家放在离第一位玩家足够远的地方
    location.append([(location[0][0]+2)%4, (location[0][1]+2)%4])
    # 打成平局前的最大移动步数为50
    for o in range(50):
        # 针对每位玩家
        for i in range(2):
            locs = location[i][:] + location[1-i][:]
            locs.append(lastmove[i])
            move = p[i].evaluate(locs) % 4
            
            # 如果在一行中朝同一个方向移动了两次，就判定为你输
            if lastmove[i] == move:
                return 1-i
            lastmove[i] = move
            
            if move == 0:
                location[i][0] -= 1
                if location[i][0] < 0:
                    location[i][0] = 0
            if move == 1:
                location[i][0] += 1
                if location[i][0] > Max[0]:
                    location[i][0] = Max[0]
            if move == 2:
                location[i][1] -= 1
                if location[i][1] < 0:
                    location[i][1] = 0
            if move == 3:
                location[i][1] += 1
                if location[i][1] > Max[1]:
                    location[i][1] = Max[1]
                    
            # 如果抓住了对方玩家，就判定为你赢
            if location[i] == location[1-i]:
                return i
    
    return -1

In [62]:
p1 = makerandomtree(5)
p2 = makerandomtree(5)
gridgame([p1, p2])

1

In [69]:
def tournament(plist):
    # 统计失败的次数
    losses = [0 for p in plist]
    
    # 每位玩家都将和其他玩家一一对抗
    for i in range(len(plist)):
        for j in range(len(plist)):
            if i == j:
                continue
            
            # 谁是胜利者？
            winner = gridgame([plist[i], plist[j]])
            # 失败得2分，打平得1分
            if winner == 0:
                losses[j] += 2
            elif winner == 1:
                losses[i] += 2
            elif winner == -1:
                losses[i] += 1
                losses[i] += 1
                pass
    
    # 对结果排序并返回
    # 获取列表的第一个元素
        def takeFirst(elem):
            return elem[0]
    z = list(zip(losses, plist))
    z.sort(key=takeFirst)
    return z

In [70]:
winner = evolve(5, 100, tournament, maxgen=50)

22
40
68
66
66
76
82
68
70
94
84
90
82
48
64
70
62
46
76
48
52
70
72
38
60
58
56
62
68
68
72
64
64
42
70
50
46
66
62
66
52
60
66
52
44
40
36
42
38
24
if
 subtract
  3
  p4
 7
 isgreater
  isgreater
   subtract
    multiply
     7
     p2
    p1
   isgreater
    add
     subtract
      isgreater
       p4
       if
        add
         add
          multiply
           if
            4
            3
            p1
           subtract
            6
            7
          if
           p3
           isgreater
            4
            p0
           p4
         2
        p2
        1
      isgreater
       p2
       add
        p1
        p4
     multiply
      p0
      multiply
       isgreater
        p0
        p0
       p4
    p4
  subtract
   p3
   p4


In [78]:
class humanplayer:
    def evaluate(self, board):
        # 得到自己的位置和其他玩家的位置
        me = tuple(board[0:2])
        others = [tuple(board[x:x+2]) for x in range(2, len(board)-1, 2)]
        # 显示游戏区域
        for i in range(4):
            for j in range(4):
                if (i,j) == me:
                    print('O', end='')
                elif (i,j) in others:
                    print('X', end='')
                else:
                    print('.', end='')
            print('')
            
        # 显示上一步，作为参考
        print('Your last move was %d' % board[len(board)-1])
        print(' 0')
        print('2 3')
        print(' 1')
        print('Enter move: ')
        
        # 不论用户输入什么内容，均直接返回
        move = int(input())
        return move

In [82]:
gridgame([winner, humanplayer()])

....
.O..
....
...X
Your last move was -1
 0
2 3
 1
Enter move: 
1
....
....
.O..
...X
Your last move was 1
 0
2 3
 1
Enter move: 
3
....
....
..O.
...X
Your last move was 3
 0
2 3
 1
Enter move: 
1
....
....
....
..OX
Your last move was 1
 0
2 3
 1
Enter move: 
2
....
....
....
.O.X
Your last move was 2
 0
2 3
 1
Enter move: 
0
....
....
.O..
...X
Your last move was 0
 0
2 3
 1
Enter move: 
3
....
....
..O.
...X
Your last move was 3
 0
2 3
 1
Enter move: 
1
....
....
....
..OX
Your last move was 1
 0
2 3
 1
Enter move: 
3


1